# Deep MNIST Classifier

## Load MNIST Data

In [33]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Start TensorFlow InteractiveSession

In [34]:
import tensorflow as tf
sess = tf.InteractiveSession()

## Build a Softmax Regression Model

In [35]:
# Placeholders
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])


In [36]:
# Variables
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

## Initialize

In [37]:
sess.run(tf.initialize_all_variables())

## Predicted Class and Cost Function

In [38]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [39]:
# cost function
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

## Train the Model

In [40]:
# steepest gradient descent
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [41]:
# Training
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

## Evaluate the Model 

In [42]:
# figure out where we predicted the correct label
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

# correct_prediction : Boolean

In [43]:
# cast to floating point numbers and then take the mean.
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [44]:
# Evaluate Accuracy on the test data
print accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})

0.9092


# Multilayer Convolutional Network

## Weight Initialization

In [16]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

## Convolution and Pooling

In [18]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

## First Convolutional Layer

In [19]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])


In [20]:
# reshape x to a 4d tensor
x_image = tf.reshape(x, [-1,28,28,1])

In [21]:
#  convolve x_image with the weight tensor, add the bias, 
# apply the ReLU function, and finally max pool
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

## Second Convolutional Layer

In [23]:
# second layer will have 64 features for each 5x5 patch.

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


## Densely Connected Layer

In [25]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

## Dropout

In [27]:
# to reduce overfitting

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## Readout Layer

In [29]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## Train and Evaluate the Model

In [ ]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.initialize_all_variables())
for i in range(2000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print "step %d, training accuracy %g"%(i, train_accuracy)
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})




In [32]:
print "test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})

test accuracy 0.9802
